In [1]:
%load_ext Cython

#pseudocode
'''
define a lattice with random configuration
each lattice site has a plane of rotations that alternates**

'''

'\ndefine a lattice with random configuration\neach lattice site has a plane of rotations that alternates**\n\n'

In [2]:
%%cython -c=-ffast-math -a
from __future__ import division

import numpy as np
import random as random
cimport numpy as np
cimport cython

from numpy cimport abs
from scipy import constants
from libcpp cimport bool
from libc.math cimport exp
from libc.math cimport sin
from libc.math cimport cos
from libc.math cimport acos
from libc.math cimport sqrt
from libc.math cimport M_PI
from libc.stdlib cimport rand

cdef extern from "limits.h":
    int RAND_MAX


def random_lattice(int N, int R, double[:] u1, double[:] u2):
    
    cdef double[::1] theta =  np.random.uniform(0,2*M_PI, N*N*R)

    L = np.array(np.zeros((N,N,R), dtype=np.ndarray))
    u1 = np.array([1/np.sqrt(2),1/np.sqrt(2),0])
    u2 = np.array([-1/np.sqrt(2),1/np.sqrt(2),0])
    
    r1 = np.array([1/np.sqrt(2),-1/np.sqrt(2),0])
    r2 = np.array([-1/np.sqrt(2),-1/np.sqrt(2),0])
    r3 = np.array([1/np.sqrt(2),1/np.sqrt(2),0])
    r4 = np.array([-1/np.sqrt(2),1/np.sqrt(2),0])
    
    for k in range(R):
        for i in range(N):
            for j in range(N):
                if i % 2 == 0:
                    if j % 2 == 0:
                        if k % 2 == 0:
                            L[i,j,k] = -1*r1
                        else:
                            L[i,j,k] = r1
                    else:
                        if k % 2 == 0:
                            L[i,j,k] = -1*r3
                        else: 
                            L[i,j,k] = r3
                else:
                    if j % 2 == 0:
                        if k % 2 == 0:
                            L[i,j,k] = -1*r2
                        else:
                            L[i,j,k] = r2
                    else: 
                        if k % 2 == 0:
                            L[i,j,k] = -1*r4
                        else:
                            L[i,j,k] = r4
  
    return L
    


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef rotation(double[::1] theta, int i, np.npy_bool c, double[:] u1, double[:] u2):
    
    cdef double C = cos(theta[i])
    cdef double S = sin(theta[i])
    cdef double t = (1-cos(theta[i]))
    

    if c == True:
        return np.array([[C+(u1[0]**2)*t, u1[0]*u1[1]*t, u1[1]*S], 
                       [u1[1]*u1[0]*t, C+(u1[1]**2)*t, -u1[0]*S], [-u1[1]*S, u1[0]*S, C]])

    else:
        return np.array([[C+(u2[0]**2)*t, u2[0]*u2[1]*t, u2[1]*S], 
                       [u2[1]*u2[0]*t, C+(u2[1]**2)*t, -u2[0]*S], [-u2[1]*S, u2[0]*S, C]])
    
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef unit_vector(vector):
    return vector / np.linalg.norm(vector)


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return acos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    

def cy_aa_step(L, beta, N, R, r1, r2, r3, r4, 
                r1_2, r2_2, r3_2, r4_2, r1_3, r2_3, 
                u1, u2):
    g = 0
    theta = np.random.uniform(0,2*M_PI, N*N*R)
    i = 0
    k = 0
    temp = False
    for _r in range(R):
        for n in range(N):
            for m in range(N):
                cy_aa_update(L, N, R, n, m, _r, beta, i, g, r1, r2, r3, r4, r1_2, r2_2, r3_2, r4_2, r1_3, r2_3, theta, u1, u2)
                if (i+1+k) % N == 0 and i != 0 and temp == False:
                    temp = True
                    k = k + 1
                else:
                    i = i + 1
                    temp = False
                g += 1
        i = 0
        k = 0
        temp = False
    return L

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef cy_aa_update(np.ndarray[:,:,::1] L, int N, int R, int n, int m, int _r, float beta, int i, int g, 
                  int[:] r1, int[:] r2, int[:] r3, int[:] r4, double[:] r1_2, 
                  double[:] r2_2, double[:] r3_2, double[:] r4_2, int[:] r1_3, int[:] r2_3, 
                  double[::1] theta, double[:] u1, double[:] u2):
    
    if (i)%2 == 0:
        c = True
    else:
        c = False
                
    r = rotation(np.asarray([theta[g]]), 0, c, u1, u2).dot(L[n,m,_r])
    
#initial
    #nearest neighbors
    cdef float H1_i = -1*(3*(L[n,m,_r].dot(r1))*(L[(n + 1)%N,m,_r].dot(r1))-(L[n,m, _r].dot(L[(n + 1)%N, m, _r])))
    cdef float H2_i = -1*(3*(L[n,m,_r].dot(r2))*(L[(n - 1)%N, m,_r].dot(r2))-(L[n,m, _r].dot(L[(n - 1)%N, m, _r])))
    cdef float H3_i = -1*(3*(L[n,m,_r].dot(r3))*(L[n, (m + 1)%N,_r].dot(r3))-(L[n,m, _r].dot(L[n, (m + 1)%N, _r])))
    cdef float H4_i = -1*(3*(L[n,m,_r].dot(r4))*(L[n, (m - 1)%N,_r].dot(r4))-(L[n,m, _r].dot(L[n, (m - 1)%N, _r])))
    cdef float H5_i = -1*(3*(L[n,m,_r].dot(r1_3))*(L[n, m, (_r - 1)%R].dot(r1_3))-(L[n,m,_r].dot(L[n, m, (_r - 1)%R])))
    cdef float H6_i = -1*(3*(L[n,m,_r].dot(r2_3))*(L[n, m, (_r + 1)%R].dot(r2_3))-(L[n,m,_r].dot(L[n, m, (_r + 1)%R])))
    #2nd nearest neighbors
    cdef float H1_2i = -1*(1/np.sqrt(2)**3)*(3*(L[n,m,_r].dot(r1_2))*(L[(n + 1)%N, (m + 1)%N,_r].dot(r1_2))-(L[n,m,_r].dot(L[(n + 1)%N, (m + 1)%N,_r])))
    cdef float H2_2i = -1*(1/np.sqrt(2)**3)*(3*(L[n,m,_r].dot(r2_2))*(L[(n + 1)%N, (m - 1)%N,_r].dot(r2_2))-(L[n,m,_r].dot(L[(n + 1)%N, (m - 1)%N,_r])))
    cdef float H3_2i = -1*(1/np.sqrt(2)**3)*(3*(L[n,m,_r].dot(r3_2))*(L[(n - 1)%N, (m + 1)%N,_r].dot(r3_2))-(L[n,m,_r].dot(L[(n - 1)%N, (m + 1)%N,_r])))
    cdef float H4_2i = -1*(1/np.sqrt(2)**3)*(3*(L[n,m,_r].dot(r4_2))*(L[(n - 1)%N, (m - 1)%N,_r].dot(r4_2))-(L[n,m,_r].dot(L[(n - 1)%N, (m - 1)%N,_r])))

#proposed
    #nearest neighbors
    cdef float H1_f = -1*(3*(r.dot(r1))*(L[(n + 1)%N,m,_r].dot(r1))-(r.dot(L[(n + 1)%N, m,_r])))
    cdef float H2_f = -1*(3*(r.dot(r2))*(L[(n - 1)%N, m,_r].dot(r2))-(r.dot(L[(n - 1)%N, m,_r])))
    cdef float H3_f = -1*(3*(r.dot(r3))*(L[n, (m + 1)%N,_r].dot(r3))-(r.dot(L[n, (m + 1)%N,_r])))
    cdef float H4_f = -1*(3*(r.dot(r4))*(L[n, (m - 1)%N,_r].dot(r4))-(r.dot(L[n, (m - 1)%N,_r])))
    cdef float H5_f = -1*(3*(r.dot(r1_3))*(L[n, m, (_r - 1)%R].dot(r1_3))-(r.dot(L[n, m, (_r - 1)%R])))
    cdef float H6_f = -1*(3*(r.dot(r2_3))*(L[n, m, (_r + 1)%R].dot(r2_3))-(r.dot(L[n, m, (_r + 1)%R])))
   
    #2nd nearest neighbors
    cdef float H1_2f = -1*(1/np.sqrt(2)**3)*(3*(r.dot(r1_2))*(L[(n + 1)%N, (m + 1)%N,_r].dot(r1_2))-(r.dot(L[(n + 1)%N, (m + 1)%N,_r])))
    cdef float H2_2f = -1*(1/np.sqrt(2)**3)*(3*(r.dot(r2_2))*(L[(n + 1)%N, (m - 1)%N,_r].dot(r2_2))-(r.dot(L[(n + 1)%N, (m - 1)%N,_r])))
    cdef float H3_2f = -1*(1/np.sqrt(2)**3)*(3*(r.dot(r3_2))*(L[(n - 1)%N, (m + 1)%N,_r].dot(r3_2))-(r.dot(L[(n - 1)%N, (m + 1)%N,_r])))
    cdef float H4_2f = -1*(1/np.sqrt(2)**3)*(3*(r.dot(r4_2))*(L[(n - 1)%N, (m - 1)%N,_r].dot(r4_2))-(r.dot(L[(n - 1)%N, (m - 1)%N,_r])))
    
#     cdef np.float64_t[:] Lp = np.array(L[n,m,_r])
#     cdef np.float64_t[:] rp = np.array(r)
#     Lp[2] = 0.0
#     rp[2] = 0.0
    
#     cdef double tl = np.array([angle_between(L[n,m], Lp)])
#     cdef double tr = np.array([angle_between(r, rp)])

#     cdef double rotp = 18.80436397

    cdef float H_i = H1_i + H2_i + H3_i + H4_i + H5_i + H6_i + H1_2i + H2_2i + H3_2i + H4_2i# + rotp*sin(tl)
    cdef float H_f = H1_f+ H2_f + H3_f + H4_f + H5_f + H6_f + H1_2f + H2_2f + H3_2f + H4_2f# + rotp*sin(tr)
    cdef float H = H_f - H_i

    if H <= 0:
        L[n, m, _r] = r 

    elif rand() < RAND_MAX * exp(-beta * H):
        L[n, m, _r] = r 
        

In [169]:
from mayavi import mlab
from mayavi.modules.glyph import Glyph
import numpy as np
%gui qt

#msplt = plt.mlab_source
@mlab.animate(delay=10)
def anim(L):
    while True:
        L=cy_aa_step(L, b1, N, R, r1, r2, r3, r4, r1_2, r2_2, r3_2, r4_2, r1_3, r2_3, u1, u2) 
        A = []
        for k in range(R):
            for i in range(N):
                for j in range(N):
                    A.append(L[j][i][k].tolist())
        U, V, W = zip(*np.stack(A))
        plt.mlab_source.set(x=X,y=Y,z=Z,u=U,v=V,w=W)  
        yield
        

N = 8
R = 2
A = []
b1 = 1.0 / .3

r1 = np.array([0,1,0], dtype = np.int32) 
r2 = np.array([0,-1,0], dtype = np.int32)
r3 = np.array([1,0,0], dtype = np.int32)
r4 = np.array([-1,0,0], dtype = np.int32)

r1_2 = np.array([1/np.sqrt(2),-1/np.sqrt(2),0])
r2_2 = np.array([-1/np.sqrt(2),-1/np.sqrt(2),0])
r3_2 = np.array([1/np.sqrt(2),1/np.sqrt(2),0])
r4_2 = np.array([-1/np.sqrt(2),1/np.sqrt(2),0])

r1_3 = np.array([0,0,1], dtype= np.int32)
r2_3 = np.array([0,0,-1], dtype= np.int32)

u1 = np.array([1/np.sqrt(2),1/np.sqrt(2),0])
u2 = np.array([-1/np.sqrt(2),1/np.sqrt(2),0])
    
L = random_lattice(N, R, u1, u2)
for k in range(R):
    for i in range(N):
        for j in range(N):
            A.append(L[j][i][k].tolist())

# X = [b for a in range(R) for b in range(N) for x in range(N)]
# Y = [x for a in range(R) for b in range(N) for x in range(N)]
# Z = [a for a in range(R) for b in range(N) for x in range(N)]

X = [b for b in range(N*2) for x in range(N)]
Y = [x for b in range(N*2) for x in range(N)]
Z = [0]*N*N*4

# for x in range(len(X)):
#     if X[x] > 7:
#         X[x] += 3
        
# for i in range(N):
#     for t in range(8):
#         X.append(i)

# for i in range(N+3, N+3+N):
#     for t in range(8):
#         X.append(i)
        
# for t in range(16):
#     for j in range(10,18):
#         Y.append(j)

N = 10
_x = [b for b in range(N*2) for x in range(N)]
_y = [x for b in range(N*2) for x in range(N)]
_z = [0]*N*N*4
L = random_lattice(N, R, u1, u2)
B = []
for k in range(R):
    for i in range(N):
        for j in range(N):
            B.append(L[j][i][k].tolist())
            
# for x in range(len(_x)):
#     if _x[x] > 9:
#         _x[x] += 3
        
# for i in range(N):
#     for t in range(10):
#         _x.append(i)

# for i in range(N+3, N+3+N):
#     for t in range(10):
#         _x.append(i)
        
# for t in range(20):
#     for j in range(10,20):
#         _y.append(j)
        
    
print (len(X))
print (len(Y))
print (len(Z))
X = tuple(X)
Y = tuple(Y)
Z = tuple(Z)


f = mlab.figure()
for b in range(len(B)):
    B[b] = [0,1,0]
U, V, W = zip(*np.stack(A))
_u, _v, _w = zip(*np.stack(B))

a = 0
b = 0
for i in range(len(X)):
    if a%2 == 0 and b%2==0:
        color = (0,0,1)
        a+=1
    elif a%2 == 1 and b%2==0:
        color = (0,1,0)
        a+=1
    elif a%2 == 0 and b%2==1:
        color = (1,0.54,0)
        a+=1
    elif a%2 == 1 and b%2==1:
        color = (1,0,1)
        a+=1
    if i % 8== 0:
        b+=1
    
# for i in range(len(X)):
#     if len(X)/2 > i:
#         color = (1,.2,.4)
#     else:
#         color = (.1,.1,.3)    
    plt = mlab.quiver3d(X[i],Y[i],Z[i],U[i],V[i],W[i], color=color, scale_factor = 1)
    plt.glyph.glyph_source.glyph_position = 'center'
    
a = 0
b = 0
for j in range(len(_x)):

    if a%2 == 0 and b%2==0:
        color = (0,0,1)
        a+=1
    elif a%2 == 1 and b%2==0:
        color = (0,1,0)
        a+=1
    elif a%2 == 0 and b%2==1:
        color = (1,0.54,0)
        a+=1
    elif a%2 == 1 and b%2==1:
        color = (1,0,1)
        a+=1
    if j % 10== 0:
        b+=1
    
    if j > len(_x)/4 and j < len(_x)/2:
        t = 2
    elif j > len(_x)/4*3:
        t = 2
    else:
        t= 0
    if a%2== 0 and b%2==0 and a%20!=0:
        nodes = mlab.quiver3d(_x[j], _y[j], _z[j], _u[j], _v[j], _w[j], color = (0,0,0))
        nodes.glyph.glyph_source.glyph_source.glyph_type = 'diamond'
        nodes.glyph.glyph_source.glyph_source.center = (-.5,-1.5,0)
        nodes.glyph.glyph_source.glyph_source.scale=2
        nodes.actor.actor.property.line_width = 1
#         nodes.glyph.glyph_source.glyph_source.rotation_angle = 45


print nodes.actor.actor.property

# print mlab.outline()
# g = Glyph()
# mlab.glyph(0)

#         mlab.outline().actor.property.color = (0,0,0)
#         mlab.outline().actor.actor.orientation = (0,0,45)
#         mlab.outline().actor.actor.position = (0,0,0)
#         mlab.outline().actor.actor.scale = (1,1,0)
    
# mlab.outline().actor.actor.orientation=(0,0,45)
# mlab.outline().actor.actor.position=(6.5,-3.5,0)
# mlab.outline().actor.actor.scale = (2,2,0)

# print mlab.outline()
# plt = mlab.quiver3d(X,Y,Z,U,V,W, color=(1,1,1),scale_factor = 0.6)
# anim(L)
mlab.show()


128
128
256


IndexError: tuple index out of range

In [ ]:
A = np.sum(L)
print A
np.linalg.norm(A, ord = 3)

In [ ]:
import numpy as np
G = [2,2,2,2,2,2]
print G[:2]

In [292]:
# from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

# Make the grid
x, y, z = np.meshgrid(np.arange(-0.8, 1, 0.2),
                      np.arange(-0.8, 1, 0.2),
                      np.arange(-0.8, 1, 0.8))

# Make the direction data for the arrows
u = np.sin(np.pi * x) * np.cos(np.pi * y) * np.cos(np.pi * z)
v = -np.cos(np.pi * x) * np.sin(np.pi * y) * np.cos(np.pi * z)
w = (np.sqrt(2.0 / 3.0) * np.cos(np.pi * x) * np.cos(np.pi * y) *
     np.sin(np.pi * z))

# Color by azimuthal angle
c = np.arctan2(v, u)
# Flatten and normalize
c = (c.ravel() - c.min()) / c.ptp()
# Repeat for each body line and two head lines
c = np.concatenate((c, np.repeat(c, 2)))
# Colormap
c = plt.cm.hsv(c)
print c
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.quiver(x, y, z, u, v, w, colors=c, length=0.1, normalize=True)
# plt.show()

[[1.         0.         0.81176542 1.        ]
 [0.         1.         0.22353062 1.        ]
 [1.         0.         0.81176542 1.        ]
 ...
 [1.         0.         0.81176542 1.        ]
 [0.         1.         0.22353062 1.        ]
 [0.         1.         0.22353062 1.        ]]
